# Script Goals:

1. Fuzzy match official county names to internal company geodatabase county names 
    (Internal county names have inconsistent spellings, have 2 counties for 1 row, etc.)

In [107]:
import pandas as pd
import geopandas as gp
import datetime as dt
import pdb
import numpy as np
import requests as req

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [108]:
usCounties = gp.read_file('zip://Source Files/UScounties/UScounties.zip')

usCounties.head()
usCounties.info()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry
0,Lake of the Woods,Minnesota,27,077,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715..."
1,Ferry,Washington,53,019,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4..."
2,Stevens,Washington,53,065,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0..."
3,Okanogan,Washington,53,047,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9..."
4,Pend Oreille,Washington,53,051,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9..."


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 3141 entries, 0 to 3140
Data columns (total 6 columns):
NAME          3141 non-null object
STATE_NAME    3141 non-null object
STATE_FIPS    3141 non-null object
CNTY_FIPS     3141 non-null object
FIPS          3141 non-null object
geometry      3141 non-null geometry
dtypes: geometry(1), object(5)
memory usage: 147.3+ KB


In [109]:
#converting fips fields to integers for joining with state table

for col in list(filter(lambda x: "FIPS" in x, usCounties.columns)):
    print(col)
    usCounties[col] = usCounties[col].astype(int)

STATE_FIPS
CNTY_FIPS
FIPS


In [110]:
stlink = 'https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696'
stTable = pd.read_html(req.get(stlink).content)[0]

In [111]:

#last row had null fips due to extra read row from html source
stTable = stTable.iloc[:-1, :]
stTable["FIPS"] = stTable["FIPS"].astype(int)
stTable.rename(columns = {"FIPS": "State Fips"}, inplace = True)
stTable.head()

,Name,Postal Code,State Fips
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6


In [112]:
usCounties = pd.merge(usCounties, stTable.iloc[:,1:], how="left", left_on = "STATE_FIPS", right_on = "State Fips")
usCounties.drop(columns ="State Fips", inplace = True)
usCounties

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry,Postal Code
0,Lake of the Woods,Minnesota,27,77,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715...",MN
1,Ferry,Washington,53,19,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4...",WA
2,Stevens,Washington,53,65,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0...",WA
3,Okanogan,Washington,53,47,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9...",WA
4,Pend Oreille,Washington,53,51,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9...",WA
5,Boundary,Idaho,16,21,16021,"POLYGON ((-117.02911 48.83808, -117.03205 48.9...",ID
6,Lincoln,Montana,30,53,30053,"POLYGON ((-116.05550 48.20848, -116.05669 48.4...",MT
7,Flathead,Montana,30,29,30029,"POLYGON ((-113.47363 47.59758, -113.63732 47.6...",MT
8,Glacier,Montana,30,35,30035,"POLYGON ((-112.18273 48.47117, -112.22915 48.4...",MT
9,Toole,Montana,30,101,30101,"POLYGON ((-111.42231 48.21776, -111.67355 48.2...",MT


In [113]:
usCounties["County Fuzzy Choices"] = usCounties["NAME"] + ", " + usCounties["Postal Code"]

In [114]:
usCounties.head()

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry,Postal Code,County Fuzzy Choices
0,Lake of the Woods,Minnesota,27,77,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715...",MN,"Lake of the Woods, MN"
1,Ferry,Washington,53,19,53019,"POLYGON ((-118.85163 47.94956, -118.84846 48.4...",WA,"Ferry, WA"
2,Stevens,Washington,53,65,53065,"POLYGON ((-117.43883 48.04412, -117.54219 48.0...",WA,"Stevens, WA"
3,Okanogan,Washington,53,47,53047,"POLYGON ((-118.97209 47.93915, -118.97406 47.9...",WA,"Okanogan, WA"
4,Pend Oreille,Washington,53,51,53051,"POLYGON ((-117.43858 48.99992, -117.03205 48.9...",WA,"Pend Oreille, WA"


# Importing BLM Master Spreadsheet & Cleaning

In [115]:
spreadsheet = pd.read_excel("Source Files/Internal Master Spreadsheet/Updated R&R & Magnum Master - BLM Leases.xlsx",
                            header = 9, usecols = 'A:P')

#dropping rows with no serial numbers from spreadsheet - these are typically rows for spacing or subtotals
spreadsheet.dropna(subset=["LEASE NO."], inplace = True)
spreadsheet.shape
spreadsheet.info()
spreadsheet.head()

(2182, 16)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2182 entries, 0 to 2564
Data columns (total 16 columns):
LEASE NO.                  2182 non-null object
PLOTTED                    2182 non-null object
GRANTEE                    2182 non-null object
SALE DATE                  2162 non-null object
EFFECTIVE DATE OF LEASE    2083 non-null object
EXPIRATION DATE            2082 non-null datetime64[ns]
ACRES                      2109 non-null float64
COUNTY                     2128 non-null object
ST                         2133 non-null object
STATUS/COMMENTS            913 non-null object
EXPIRATION YEAR            2077 non-null float64
TOWNSHIP                   1332 non-null object
RANGE                      1331 non-null object
DESCRIPTION I              1930 non-null object
DESCRIPTION II             1868 non-null object
DESCRIPTION III            844 non-null object
dtypes: datetime64[ns](1), float64(2), object(13)
memory usage: 289.8+ KB


,LEASE NO.,PLOTTED,GRANTEE,SALE DATE,EFFECTIVE DATE OF LEASE,EXPIRATION DATE,ACRES,COUNTY,ST,STATUS/COMMENTS,EXPIRATION YEAR,TOWNSHIP,RANGE,DESCRIPTION I,DESCRIPTION II,DESCRIPTION III
0,TXNM096140,NO,R&R Royalty,1995-10-19 00:00:00,1995-12-01 00:00:00,2005-12-01,NaN,MONTGOMERY,TX,Expired 2005,2005.0,NaN,NaN,ASSIGNMENT REC'D FROM DAN,"TRACTS J22, J22A, J22B",NaN
2,TXNM103231,NO,R&R Royalty,1999-05-13 00:00:00,1999-07-01 00:00:00,2009-07-01,NaN,HOUSTON,TX,Expired 2009 - Bought from Dan Gonzales,2009.0,NaN,NaN,SAM HOUSTON NATL FOREST,ASSIGNMENT REC'D FROM DAN,NaN
4,TXNM091527,NO,R&R Royalty,1993-07-21 00:00:00,1993-09-01 00:00:00,2003-09-01,NaN,KLEBERG,TX,Expired 2003,2003.0,NaN,NaN,KINGSVILLE NAS,J MINDIOLOA SURVEY,LOTS & TRACTS IN SEC 23 & 30
6,TXNM102849,NO,R&R Royalty,1999-04-21 00:00:00,1999-06-01 00:00:00,2009-06-01,NaN,KLEBERG,TX,Expired 2009,2009.0,NaN,NaN,TEXAS A&M UNIVERSITY,"LOTS 2-8, BLK 5, KING ADDITION-EX 47.75 A",NaN
8,TXNM103276,NO,R&R Royalty,1999-07-21 00:00:00,1999-09-01 00:00:00,2009-09-01,NaN,HOUSTON,TX,Expired 2009 - sold,2009.0,NaN,NaN,DAVY CROCKETT NATL FOREST,TRACT K-1g,NaN


In [116]:
spreadsheet["County for Matching"] = spreadsheet["COUNTY"] + ", " + spreadsheet["ST"]

In [117]:
pd.options.display.max_rows=300
spreadsheet["County for Matching"].value_counts().sort_index()

ADAMS, MS                               9
ALCONA, MI                              6
ALFALFA, OK                             1
ALLEGAN, MI                            16
AMITE, MS                              19
AVOYELLES, LA                           5
BEAUREGARD, LA                          1
BEAVER, OK                              4
BIBB, AL                                5
BIENVILLE PARISH, LA                    1
BILLINGS, ND                            4
BLAINE, MT                              3
BLOUNT, AL                              4
BOSSIER PARISH, LA                      2
BOTTINEAU, ND                           1
BOWMAN, ND                             15
BRADFORD, PA                            5
BURLESON, TX                            3
Burleson, TX                            1
CADDO & DESOTO, LA                      1
CADDO , OK                              1
CADDO PARISH , LA                       1
CADDO PARISH, LA                        5
CALCASIEU PARISH, LA              

## Fuzzy Matching County Names

In [118]:
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [119]:
def fuzzyMatchCustom(query, choices, User_score_cutoff = 0):
    '''
    This function will fuzzy matching to find matches for lease serial numbers in our master spreadsheet to BLM lR2000 
    database serial numbers or counties with official county names
    
    '''
    
    #assigingn an 88 score cutoff to avoid false matches
    match = process.extractOne(query, choices, score_cutoff=User_score_cutoff)
    if type(match) == type(None):
        return "No Match"
    else:
        return match[0]

In [120]:
isRun = input("This cell is computionally expensive/time consuming, run?")
if isRun == "yes":
    spreadsheet["Fuzzy Matched County"] = spreadsheet["County for Matching"].apply(lambda x: fuzzyMatchCustom(str(x), usCounties["County Fuzzy Choices"], User_score_cutoff = 88))
else:
    pass



This cell is computionally expensive/time consuming, run?n


### Exporting Full Spreadsheet to Output List

In [121]:
isRun = input("This cell would overwrite exported previously exported data, run?")
if isRun == "yes":
    spreadsheet.to_excel("Output Lists/BLM Master - Fuzzy Match Counties.xlsx", index = False)
else:
    pass

This cell would overwrite exported previously exported data, run?n


In [122]:
spreadsheet=pd.read_excel("Output Lists/BLM Master - Fuzzy Match Counties.xlsx")

## Analyzing Records that Did not Have County Match on Active Lease Dataset

In [123]:
spreadsheet[spreadsheet["EXPIRATION DATE"]>'2021-07-01']["Fuzzy Matched County"].value_counts().sort_index()

Adams, MS              5
Alcona, MI             5
Alfalfa, OK            1
Allegan, MI           16
Amite, MS             18
Avoyelles, LA          5
Bibb, AL               1
Billings, ND           2
Blaine, MT             3
Bossier, LA            1
Bottineau, ND          1
Bowman, ND            15
Bradford, PA           5
Burleson, TX           2
Caddo, LA              4
Caldwell, LA           1
Campbell, WY          29
Carbon, MT             9
Carbon, WY             1
Catahoula, LA          2
Chaves, NM            16
Chouteau, MT           1
Claiborne, LA          1
Clarke, AL             1
Cleburne, AR          11
Converse, WY          17
Covington, AL         14
Covington, MS          5
Crook, WY              1
Daniels, MT           13
Dawson, MT             6
De Soto, LA            1
Dewey, OK              1
Eddy, NM              23
Ellis, OK              2
Emery, UT              1
Escambia, AL           6
Eureka, NV             2
Fall River, SD        24
Fallon, MT             6


In [124]:
noMatchesActiveLeases = spreadsheet[(spreadsheet["EXPIRATION DATE"]>'2021-07-01') & (spreadsheet["Fuzzy Matched County"] == "No Match")]

noMatchesActiveLeases

,LEASE NO.,PLOTTED,GRANTEE,SALE DATE,EFFECTIVE DATE OF LEASE,EXPIRATION DATE,ACRES,COUNTY,ST,STATUS/COMMENTS,EXPIRATION YEAR,TOWNSHIP,RANGE,DESCRIPTION I,DESCRIPTION II,DESCRIPTION III,County for Matching,Fuzzy Matched County
1580,LAES57906,YES,R&R Royalty,2014-03-20 00:00:00,2014-05-01,2024-05-01,1740.0,WEBSTER/BOSSIER PARISH,LA,NaN,2024.0,18N,10W,MERIDIAN LOUSIANA,SEE ATTACHED PAGE ON ACTUAL LEASE FOR SECTIONS.,NaN,"WEBSTER/BOSSIER , LA",No Match


### Noticed The Word "Parish" causes no match to be found

In [125]:
testString = noMatchesActiveLeases.iloc[0]["County for Matching"]

process.extract(testString, usCounties["County Fuzzy Choices"])

[('La Crosse, WI', 86, 382),
 ('Webster, IA', 86, 528),
 ('La Salle, IL', 86, 653),
 ('Webster, NE', 86, 942),
 ('Webster, WV', 86, 1314)]

In [126]:
process.extract(testString.replace("PARISH", ""), usCounties["County Fuzzy Choices"])

[('La Crosse, WI', 86, 382),
 ('Webster, IA', 86, 528),
 ('La Salle, IL', 86, 653),
 ('Webster, NE', 86, 942),
 ('Webster, WV', 86, 1314)]

## Rerunning Fuzzy Match After Stripping Parish From County Names

##### Stripping words: Parishes, counties, county, parish

In [127]:
spreadsheet["County for Matching"] = spreadsheet["County for Matching"].apply(lambda x: str(x).upper().replace('PARISHES',""))
spreadsheet["County for Matching"] = spreadsheet["County for Matching"].apply(lambda x: str(x).upper().replace('PARISH',""))
spreadsheet["County for Matching"] = spreadsheet["County for Matching"].apply(lambda x: str(x).upper().replace('COUNTIES',""))
spreadsheet["County for Matching"] = spreadsheet["County for Matching"].apply(lambda x: str(x).upper().replace('COUNTY',""))

In [128]:
isRun = input("This cell is computionally expensive/time consuming, run?")
if isRun == "yes":
    spreadsheet["Fuzzy Matched County"] = spreadsheet["County for Matching"].apply(lambda x: fuzzyMatchCustom(str(x), usCounties["County Fuzzy Choices"], User_score_cutoff = 88))
else:
    pass


This cell is computionally expensive/time consuming, run?n


In [129]:
spreadsheet[spreadsheet["Fuzzy Matched County"]=="No Match"]

,LEASE NO.,PLOTTED,GRANTEE,SALE DATE,EFFECTIVE DATE OF LEASE,EXPIRATION DATE,ACRES,COUNTY,ST,STATUS/COMMENTS,EXPIRATION YEAR,TOWNSHIP,RANGE,DESCRIPTION I,DESCRIPTION II,DESCRIPTION III,County for Matching,Fuzzy Matched County
89,LAES51351,NO,R&R Royalty,2002-03-28 00:00:00,2002-05-01,2012-05-01,273.21,CADDO & DESOTO,LA,Expired 2012 - Part of Ha Ra Suh - OPR Encore/...,2012.0,NaN,NaN,WALLACE LAKE,SEE NOTICE,"SIX (6) TRACTS, SEE NOTICE","CADDO & DESOTO, LA",No Match
943,LAES56467,YES,R&R Royalty,2010-09-16 00:00:00,2010-12-01,2020-12-01,585.15,Natchitoches & Winn Parishes,LA,EXPIRED 2020. Transfer of Operating Rights to ...,2020.0,13N,6W,"MERIDIAN LA, T13N, R6W","Sec.10, E2SWNW,E2W2SWNW,N2SW,SWSW,SE; Sec.11, ...",Subject to F.S. Controlled Surface use stipula...,"NATCHITOCHES & WINN , LA",No Match
944,LAES56468,YES,R&R Royalty,2010-09-16 00:00:00,2010-12-01,2020-12-01,433.13,Natchitoches & Winn Parishes,LA,EXPIRED 2020. Transfer of Operating Rights to ...,2020.0,13N,6W,"MERIDIAN LA, T13N, R6W","Sec.12, W2NE, NW,NESW,SWSW,N2SE,SWSE",Subject to F.S. No Surface Occupancy Stipulati...,"NATCHITOCHES & WINN , LA",No Match
945,LAES56469,YES,R&R Royalty,2010-09-16 00:00:00,2010-12-01,2020-12-01,432.59,Natchitoches & Winn Parishes,LA,EXPIRED 2020,2020.0,13N,6W,"MERIDIAN LA, T13N, R6W","Sec. 13, E2,NWSW,S2SW",Subject to F.S. No Surface Occupancy Stipulati...,"NATCHITOCHES & WINN , LA",No Match
946,LAES56470,YES,R&R Royalty,2010-09-16 00:00:00,2010-12-01,2020-12-01,378.49,Natchitoches & Winn Parishes,LA,EXPIRED 2020,2020.0,13N,6W,"MERIDIAN LA, T13N, R6W","Sec. 14, S2S2, NESE; Sec.15, N2NWNW; Sec.23, N...",Subject to F.S. Controlled Surface Use Stipula...,"NATCHITOCHES & WINN , LA",No Match
947,LAES56471,YES,R&R Royalty,2010-09-16 00:00:00,2010-12-01,2020-12-01,627.00,Natchitoches & Winn Parishes,LA,EXPIRED 2020,2020.0,13N,6W,"MERIDIAN LA, T13N, R6W","Sec. 24, All",Subject to F.S. Controlled Surface Use Stipula...,"NATCHITOCHES & WINN , LA",No Match
948,LAES56472,YES,R&R Royalty,2010-09-16 00:00:00,2010-12-01,2020-12-01,632.29,Natchitoches & Winn Parishes,LA,EXPIRED 2020,2020.0,13N,6W,"MERIDIAN LA, T13N, R6W","Sec. 25, All",Subject to F.S. Controlled Surface Use Stipula...,"NATCHITOCHES & WINN , LA",No Match
949,LAES56473,YES,R&R Royalty,2010-09-16 00:00:00,2010-12-01,2020-12-01,517.26,Natchitoches & Winn Parishes,LA,EXPIRED 2020,2020.0,13N,6W,"MERIDIAN LA, T13N, R6W","Sec.26, E2, SENW,SW",Subject to F.S. Controlled Surface Use Stipula...,"NATCHITOCHES & WINN , LA",No Match
950,LAES56474,YES,R&R Royalty,2010-09-16 00:00:00,2010-12-01,2020-12-01,360.00,Natchitoches & Winn Parishes,LA,EXPIRED 2020,2020.0,13N,6W,"MERIDIAN LA, T13N, R6W","Sec.27, SENE, S2",Subject to F.S Controlled Surface Use Stipulat...,"NATCHITOCHES & WINN , LA",No Match
980,LAES56548,YES,R&R Royalty,2010-12-09 00:00:00,2011-02-01,2021-02-01,278.82,Winn & Natchitoches Parishes,LA,NaN,2021.0,10N,5W,"MERIDIAN LOUISIANA, T10N, R5W","Sec.8, Tract C-107 in SESW; S2SE; Sec.19 N2SE;...",Subject to F.S. No Surface Occupancy Stipulati...,"WINN & NATCHITOCHES , LA",No Match


### Exporting Full Spreadsheet to Output List

In [130]:
spreadsheet.to_excel("Output Lists/BLM Master - Fuzzy Match Counties.xlsx", index = False)

In [131]:
spreadsheet=pd.read_excel("Output Lists/BLM Master - Fuzzy Match Counties.xlsx")

## Joining County FIPS

In [132]:
usCounties.head(1)
spreadsheet.head(1)

,NAME,STATE_NAME,STATE_FIPS,CNTY_FIPS,FIPS,geometry,Postal Code,County Fuzzy Choices
0,Lake of the Woods,Minnesota,27,77,27077,"POLYGON ((-95.34283 48.54668, -95.34105 48.715...",MN,"Lake of the Woods, MN"


,LEASE NO.,PLOTTED,GRANTEE,SALE DATE,EFFECTIVE DATE OF LEASE,EXPIRATION DATE,ACRES,COUNTY,ST,STATUS/COMMENTS,EXPIRATION YEAR,TOWNSHIP,RANGE,DESCRIPTION I,DESCRIPTION II,DESCRIPTION III,County for Matching,Fuzzy Matched County
0,TXNM096140,NO,R&R Royalty,1995-10-19 00:00:00,1995-12-01,2005-12-01,NaN,MONTGOMERY,TX,Expired 2005,2005.0,NaN,NaN,ASSIGNMENT REC'D FROM DAN,"TRACTS J22, J22A, J22B",NaN,"MONTGOMERY , TX","Montgomery, TX"


In [133]:
spreadsheet = pd.merge(spreadsheet, usCounties[["County Fuzzy Choices", "FIPS"]], how = "left", left_on = "Fuzzy Matched County", right_on = "County Fuzzy Choices")

### Exporting Full Spreadsheet to Output List

In [134]:
spreadsheet.to_excel("Output Lists/BLM Master with FIPS.xlsx", index = False)

In [135]:
spreadsheet.head()

,LEASE NO.,PLOTTED,GRANTEE,SALE DATE,EFFECTIVE DATE OF LEASE,EXPIRATION DATE,ACRES,COUNTY,ST,STATUS/COMMENTS,EXPIRATION YEAR,TOWNSHIP,RANGE,DESCRIPTION I,DESCRIPTION II,DESCRIPTION III,County for Matching,Fuzzy Matched County,County Fuzzy Choices,FIPS
0,TXNM096140,NO,R&R Royalty,1995-10-19 00:00:00,1995-12-01,2005-12-01,NaN,MONTGOMERY,TX,Expired 2005,2005.0,NaN,NaN,ASSIGNMENT REC'D FROM DAN,"TRACTS J22, J22A, J22B",NaN,"MONTGOMERY , TX","Montgomery, TX","Montgomery, TX",48339.0
1,TXNM103231,NO,R&R Royalty,1999-05-13 00:00:00,1999-07-01,2009-07-01,NaN,HOUSTON,TX,Expired 2009 - Bought from Dan Gonzales,2009.0,NaN,NaN,SAM HOUSTON NATL FOREST,ASSIGNMENT REC'D FROM DAN,NaN,"HOUSTON , TX","Houston, TX","Houston, TX",48225.0
2,TXNM091527,NO,R&R Royalty,1993-07-21 00:00:00,1993-09-01,2003-09-01,NaN,KLEBERG,TX,Expired 2003,2003.0,NaN,NaN,KINGSVILLE NAS,J MINDIOLOA SURVEY,LOTS & TRACTS IN SEC 23 & 30,"KLEBERG, TX","Kleberg, TX","Kleberg, TX",48273.0
3,TXNM102849,NO,R&R Royalty,1999-04-21 00:00:00,1999-06-01,2009-06-01,NaN,KLEBERG,TX,Expired 2009,2009.0,NaN,NaN,TEXAS A&M UNIVERSITY,"LOTS 2-8, BLK 5, KING ADDITION-EX 47.75 A",NaN,"KLEBERG , TX","Kleberg, TX","Kleberg, TX",48273.0
4,TXNM103276,NO,R&R Royalty,1999-07-21 00:00:00,1999-09-01,2009-09-01,NaN,HOUSTON,TX,Expired 2009 - sold,2009.0,NaN,NaN,DAVY CROCKETT NATL FOREST,TRACT K-1g,NaN,"HOUSTON , TX","Houston, TX","Houston, TX",48225.0


# Joining Manipulated Master Spreadsheet with GIS Shapefile for Tableau

In [136]:
shapefile = gp.read_file("zip://Source Files/Master GIS Shapefile/BLM Master 7-2021.zip")

shapefile

,lot_no,LEASE_NO,GRANTEE,ACRES,COUNTY,ST,Est_Bonus,EFFECTIVE_,SALE_DATE,EXPIRATION,GeneralLoc,EXPIRATI_1,STATUS_COM,ObjectID,geometry
0,72266,WY-2020-12-0588,R&R Royalty,840.000,Niobrara,WY,0,None,2020-12-15,None,None,0.0,LEASE NOT ISSUED,1774,"POLYGON Z ((-11639970.496 5340973.580 0.000, -..."
1,72267,WY-2020-12-0592,R&R Royalty,1280.000,Niobrara,WY,0,None,2020-12-15,None,None,0.0,LEASE NOT ISSUED,1775,"POLYGON Z ((-11635543.812 5338747.382 0.000, -..."
2,72268,WY-2020-12-0595,R&R Royalty,960.000,Niobrara,WY,0,None,2020-12-15,None,None,0.0,LEASE NOT ISSUED,1776,"POLYGON Z ((-11633336.189 5338743.869 0.000, -..."
3,72262,WY-2020-12-0579,R&R Royalty,641.810,Niobrara,WY,0,None,2020-12-15,None,None,0.0,LEASE NOT ISSUED,1773,"POLYGON Z ((-11637213.500 5343726.602 0.000, -..."
4,72312,WY-2020-12-0432,R&R Royalty,307.800,Converse,WY,0,None,2020-12-15,None,None,0.0,LEASE NOT ISSUED,1772,"POLYGON Z ((-11723601.654 5349650.980 0.000, -..."
5,72346,WY-2020-12-0418,R&R Royalty,160.000,Converse,WY,0,None,2020-12-15,None,None,0.0,LEASE NOT ISSUED,1771,"POLYGON Z ((-11726899.534 5349637.178 0.000, -..."
6,72339,WY-2020-12-0413,R&R Royalty,160.000,Converse,WY,0,None,2020-12-15,None,None,0.0,LEASE NOT ISSUED,1770,"POLYGON Z ((-11727486.350 5353533.064 0.000, -..."
7,72495,WY-2020-12-6960,R&R Royalty,640.000,Sublette,WY,0,None,2020-12-17,None,None,0.0,LEASE NOT ISSUED,1798,"POLYGON Z ((-12224649.873 5248083.475 0.000, -..."
8,72294,WY-2020-12-6868,R&R Royalty,40.000,Converse,WY,0,None,2020-12-15,None,None,0.0,LEASE NOT ISSUED,1793,"POLYGON Z ((-11683378.069 5338603.633 0.000, -..."
9,72301,WY-2020-12-6872,R&R Royalty,160.650,Converse,WY,0,None,2020-12-15,None,None,0.0,LEASE NOT ISSUED,1794,"POLYGON Z ((-11707422.084 5357974.226 0.000, -..."


In [137]:
shapeTableau = pd.merge(left = shapefile[["LEASE_NO", "geometry"]], right = spreadsheet, left_on = "LEASE_NO", right_on = "LEASE NO.")

In [138]:
shapeTableau

,LEASE_NO,geometry,LEASE NO.,PLOTTED,GRANTEE,SALE DATE,EFFECTIVE DATE OF LEASE,EXPIRATION DATE,ACRES,COUNTY,...,EXPIRATION YEAR,TOWNSHIP,RANGE,DESCRIPTION I,DESCRIPTION II,DESCRIPTION III,County for Matching,Fuzzy Matched County,County Fuzzy Choices,FIPS
0,WY-2020-12-0588,"POLYGON Z ((-11639970.496 5340973.580 0.000, -...",WY-2020-12-0588,NO,R&R Royalty,2020-12-15 10:08:00,NaT,NaT,840.00,Niobrara,...,NaN,37N,64W,"T. 37 N., R. 64 W., 6TH PM","Sec. 17 ALL;Sec. 18 NE, NESE",NaN,"NIOBRARA, WY","Niobrara, WY","Niobrara, WY",56027.0
1,WY-2020-12-0592,"POLYGON Z ((-11635543.812 5338747.382 0.000, -...",WY-2020-12-0592,NO,R&R Royalty,2020-12-15 10:10:00,NaT,NaT,1280.00,Niobrara,...,NaN,37N,64W,"T. 37 N., R. 64 W., 6TH PM",Sec. 21 ALL;Sec. 22 ALL,NaN,"NIOBRARA, WY","Niobrara, WY","Niobrara, WY",56027.0
2,WY-2020-12-0595,"POLYGON Z ((-11633336.189 5338743.869 0.000, -...",WY-2020-12-0595,NO,R&R Royalty,2020-12-15 10:12:00,NaT,NaT,960.00,Niobrara,...,NaN,37N,64W,"T. 37 N., R. 64 W., 6TH PM",Sec. 23 ALL;Sec. 26 N2,NaN,"NIOBRARA, WY","Niobrara, WY","Niobrara, WY",56027.0
3,WY-2020-12-0579,"POLYGON Z ((-11637213.500 5343726.602 0.000, -...",WY-2020-12-0579,NO,R&R Royalty,2020-12-15 10:00:00,NaT,NaT,641.81,Niobrara,...,NaN,37N,64W,"T. 37 N., R. 64 W., 6TH PM","Sec. 4 LOTS 1-4;Sec. 4 S2N2, S2",NaN,"NIOBRARA, WY","Niobrara, WY","Niobrara, WY",56027.0
4,WY-2020-12-0432,"POLYGON Z ((-11723601.654 5349650.980 0.000, -...",WY-2020-12-0432,NO,R&R Royalty,2020-12-15 11:40:00,NaT,NaT,307.80,Converse,...,NaN,38N,70W,"T. 38 N., R. 70 W., 6TH PM",Sec. 30 LOTS 1-4;Sec. 30 E2W2,NaN,"CONVERSE, WY","Converse, WY","Converse, WY",56009.0
5,WY-2020-12-0418,"POLYGON Z ((-11726899.534 5349637.178 0.000, -...",WY-2020-12-0418,NO,R&R Royalty,2020-12-15 12:48:00,NaT,NaT,160.00,Converse,...,NaN,38N,71W,"T. 38 N., R. 71 W., 6TH PM",Sec. 26 NE,NaN,"CONVERSE, WY","Converse, WY","Converse, WY",56009.0
6,WY-2020-12-0413,"POLYGON Z ((-11727486.350 5353533.064 0.000, -...",WY-2020-12-0413,NO,R&R Royalty,2020-12-15 12:34:00,NaT,NaT,160.00,Converse,...,NaN,38N,71W,"T. 38 N., R. 71 W., 6TH PM",Sec. 14 NW,NaN,"CONVERSE, WY","Converse, WY","Converse, WY",56009.0
7,WY-2020-12-6960,"POLYGON Z ((-12224649.873 5248083.475 0.000, -...",WY-2020-12-6960,NO,R&R Royalty,2020-12-17 10:46:00,NaT,NaT,640.00,Sublette,...,NaN,30N,109W,"T. 30 N., R. 109 W., 6TH PM",Sec. 15 ALL,NaN,"SUBLETTE, WY","Sublette, WY","Sublette, WY",56035.0
8,WY-2020-12-6868,"POLYGON Z ((-11683378.069 5338603.633 0.000, -...",WY-2020-12-6868,NO,R&R Royalty,2020-12-15 11:04:00,NaT,NaT,40.00,Converse,...,NaN,37N,67W,"T. 37 N., R. 67 W., 6TH PM",Sec. 18 SESW,NaN,"CONVERSE, WY","Converse, WY","Converse, WY",56009.0
9,WY-2020-12-6872,"POLYGON Z ((-11707422.084 5357974.226 0.000, -...",WY-2020-12-6872,NO,R&R Royalty,2020-12-15 11:18:00,NaT,NaT,160.65,Converse,...,NaN,38N,69W,"T. 38 N., R. 69 W., 6TH PM","Sec. 5 LOTS 3, 4;Sec. 5 S2NW",NaN,"CONVERSE, WY","Converse, WY","Converse, WY",56009.0


In [139]:
#converting shapefile columns to strings from datetime for esri compatibility

shapeTableau.info()

for col in list(filter(lambda x: "DATE" in x.upper(), shapeTableau.columns)):
    shapeTableau[col] = shapeTableau[col].astype(str)

shapeTableau.info()



<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1872 entries, 0 to 1871
Data columns (total 22 columns):
LEASE_NO                   1872 non-null object
geometry                   1872 non-null geometry
LEASE NO.                  1872 non-null object
PLOTTED                    1872 non-null object
GRANTEE                    1872 non-null object
SALE DATE                  1854 non-null object
EFFECTIVE DATE OF LEASE    1829 non-null datetime64[ns]
EXPIRATION DATE            1829 non-null datetime64[ns]
ACRES                      1868 non-null float64
COUNTY                     1870 non-null object
ST                         1870 non-null object
STATUS/COMMENTS            649 non-null object
EXPIRATION YEAR            1827 non-null float64
TOWNSHIP                   1301 non-null object
RANGE                      1300 non-null object
DESCRIPTION I              1696 non-null object
DESCRIPTION II             1634 non-null object
DESCRIPTION III            652 non-null object
Cou

### Dropping duplicate field, Lease No. after join

In [140]:
shapeTableau.head(1)
shapeTableau.drop(columns="LEASE NO.", inplace = True)
shapeTableau.head()

,LEASE_NO,geometry,LEASE NO.,PLOTTED,GRANTEE,SALE DATE,EFFECTIVE DATE OF LEASE,EXPIRATION DATE,ACRES,COUNTY,...,EXPIRATION YEAR,TOWNSHIP,RANGE,DESCRIPTION I,DESCRIPTION II,DESCRIPTION III,County for Matching,Fuzzy Matched County,County Fuzzy Choices,FIPS
0,WY-2020-12-0588,"POLYGON Z ((-11639970.496 5340973.580 0.000, -...",WY-2020-12-0588,NO,R&R Royalty,2020-12-15 10:08:00,NaT,NaT,840.0,Niobrara,...,NaN,37N,64W,"T. 37 N., R. 64 W., 6TH PM","Sec. 17 ALL;Sec. 18 NE, NESE",NaN,"NIOBRARA, WY","Niobrara, WY","Niobrara, WY",56027.0


,LEASE_NO,geometry,PLOTTED,GRANTEE,SALE DATE,EFFECTIVE DATE OF LEASE,EXPIRATION DATE,ACRES,COUNTY,ST,...,EXPIRATION YEAR,TOWNSHIP,RANGE,DESCRIPTION I,DESCRIPTION II,DESCRIPTION III,County for Matching,Fuzzy Matched County,County Fuzzy Choices,FIPS
0,WY-2020-12-0588,"POLYGON Z ((-11639970.496 5340973.580 0.000, -...",NO,R&R Royalty,2020-12-15 10:08:00,NaT,NaT,840.00,Niobrara,WY,...,NaN,37N,64W,"T. 37 N., R. 64 W., 6TH PM","Sec. 17 ALL;Sec. 18 NE, NESE",NaN,"NIOBRARA, WY","Niobrara, WY","Niobrara, WY",56027.0
1,WY-2020-12-0592,"POLYGON Z ((-11635543.812 5338747.382 0.000, -...",NO,R&R Royalty,2020-12-15 10:10:00,NaT,NaT,1280.00,Niobrara,WY,...,NaN,37N,64W,"T. 37 N., R. 64 W., 6TH PM",Sec. 21 ALL;Sec. 22 ALL,NaN,"NIOBRARA, WY","Niobrara, WY","Niobrara, WY",56027.0
2,WY-2020-12-0595,"POLYGON Z ((-11633336.189 5338743.869 0.000, -...",NO,R&R Royalty,2020-12-15 10:12:00,NaT,NaT,960.00,Niobrara,WY,...,NaN,37N,64W,"T. 37 N., R. 64 W., 6TH PM",Sec. 23 ALL;Sec. 26 N2,NaN,"NIOBRARA, WY","Niobrara, WY","Niobrara, WY",56027.0
3,WY-2020-12-0579,"POLYGON Z ((-11637213.500 5343726.602 0.000, -...",NO,R&R Royalty,2020-12-15 10:00:00,NaT,NaT,641.81,Niobrara,WY,...,NaN,37N,64W,"T. 37 N., R. 64 W., 6TH PM","Sec. 4 LOTS 1-4;Sec. 4 S2N2, S2",NaN,"NIOBRARA, WY","Niobrara, WY","Niobrara, WY",56027.0
4,WY-2020-12-0432,"POLYGON Z ((-11723601.654 5349650.980 0.000, -...",NO,R&R Royalty,2020-12-15 11:40:00,NaT,NaT,307.80,Converse,WY,...,NaN,38N,70W,"T. 38 N., R. 70 W., 6TH PM",Sec. 30 LOTS 1-4;Sec. 30 E2W2,NaN,"CONVERSE, WY","Converse, WY","Converse, WY",56009.0


In [145]:
shapeTableau.to_file("Output Lists/PublicTableauShape.shp")

# Internal Version with Investment Info

In [ ]:
fullMaster = pd.read_excel("Source Files/Internal Master Spreadsheet/Updated R&R & Magnum Master - BLM Leases.xlsx",
                            header = 9)

In [ ]:
fullMaster.columns



In [ ]:
colsForInvestmentInfo = ['LEASE NO.','DUE AT SALE', 'DUE AFTER SALE (REFUNDED)', 'TOTAL  PAID',
       'AMOUNT OF RENTAL', 'YEARS RENEWED', ' RENTALS PAID UP TO DATE',
       'TOTAL INVESTMENT (BONUS + RENTALS)', 'Estimated Bonus $/Acre',]

In [ ]:
investmentData = fullMaster[colsForInvestmentInfo]

In [ ]:
shapeTableau

In [ ]:
internalShape = pd.merge(shapeTableau, investmentData, how = "inner", left_on = "LEASE_NO", right_on= 'LEASE NO.')

internalShape

In [ ]:
internalShape.columns

In [ ]:
#dropping diplicate lease number fields
internalShape.drop(columns = "LEASE NO.", inplace = True)

In [ ]:
internalShape

In [ ]:
internalShape.to_file('testshape.shp')

In [ ]:
#internalShape.to_file("Output Lists/INTERNAL_TableauShape.shp")